In [13]:
# This script is used for testing all adaptations and changes made

In [23]:
import pandas as pd
import numpy as np

from scipy import linalg
from scipy import stats

In [24]:
# Load data for change detection as a pandas data frame.
infys_df =  pd.read_csv("../data/iMAD/iMAD_Data_13_SEF.csv") #need to change the dataset 
infys_df.head()

,Unnamed: 0,Cycle,Cluster_ID,AvgDbh,AvgCrownDiameter,AvgCrownHeight,AvgCrownArea,SpeciesCount,AvgTreeHeight,TreeCount,J,X,Y
0,1,1,77652,16.103571,3.937736,3.367925,15.409993,8.0,8.555357,56.0,0.440017,-92.433389,15.588935
1,2,1,28451,10.380952,2.209524,0.000000,3.956920,2.0,2.909524,21.0,0.334649,-99.546778,26.072916
2,3,1,55242,14.996429,4.439683,4.557937,17.199388,5.0,5.458333,84.0,0.505430,-104.218713,20.403546
3,4,1,20795,11.600000,3.169565,4.286957,8.243627,5.0,4.493548,31.0,0.770242,-108.957083,27.390907
4,5,1,53236,12.194444,4.886047,5.753488,20.022403,6.0,7.605556,54.0,0.841231,-98.775463,20.791278


In [25]:
# Create new ids by concatenating 'Conglomerado' and 'Sitio'.
    ## don't think I need to do that
    ## Use existing ID of Metadataset called Cluster_ID -> should be able to leave that one out


# As we want to compare cycle 1 vs cycle 2 lets separate them.   
## Stays the same for long dataformat
## might figure out how to 
infys_df_c1 = infys_df[infys_df['Cycle'] == 1]
infys_df_c2 = infys_df[infys_df['Cycle'] == 3]

In [26]:
# Find intersection of new ID variable: 'cng_sit', so obs. that are present in both cycles.
s1 = infys_df_c1['Cluster_ID']
s2 = infys_df_c2['Cluster_ID']
cluster_id_inter = pd.Series(list(set(s1).intersection(set(s2))))
## might have already done this in R by using my filter(muestreado1 == 1 & muestreado2 == 1)



In [27]:
# Filter out obs. that dont appear in cng_sit_inter.
infys_df_c1 = infys_df_c1[infys_df_c1['Cluster_ID'].isin(cluster_id_inter)]
infys_df_c2 = infys_df_c2[infys_df_c2['Cluster_ID'].isin(cluster_id_inter)]

In [28]:
# Select only variables to be utilized in change detection.
vars = ["SpeciesCount", "TreeCount", "J", "AvgTreeHeight", "AvgDbh", "AvgCrownDiameter", "AvgCrownHeight", "AvgCrownArea"]
nvars = len(vars)
infys_df_c1_c = infys_df_c1[infys_df_c1.columns.intersection(vars)]
infys_df_c2_c = infys_df_c2[infys_df_c2.columns.intersection(vars)]

c1_c_nans = infys_df_c1_c.isna().any(axis=1).to_numpy()
c2_c_nans = infys_df_c2_c.isna().any(axis=1).to_numpy()

In [29]:
# Create change data matrix, missing values mask and initial change weights.
dm = np.zeros((2 * nvars, infys_df_c1_c.shape[0]))
dm[0:nvars] = np.transpose(infys_df_c1_c.to_numpy())
dm[nvars:] = np.transpose(infys_df_c2_c.to_numpy())

nodataidx = c1_c_nans | c2_c_nans
gooddataidx = nodataidx == False
dm = dm[:, gooddataidx]
ngood = np.sum(gooddataidx)

In [30]:
# Change detection iterations.

# iteration of MAD   
wt = np.ones(int(ngood))
delta = 1.0
oldrho = np.zeros(nvars)
iter = 0
 
while (delta > 0.01) and (iter < 35): 
    print(iter)
    # Weighted covariance matrices and means.
    sumw = np.sum(wt)
    means = np.average(dm,axis=1, weights=wt)
    dmc = dm - means[:,np.newaxis]
    dmc = np.multiply(dmc,np.sqrt(wt))
    sigma = np.dot(dmc,dmc.T)/sumw
   
    s11 = sigma[0:nvars, 0:nvars]
    s22 = sigma[nvars:, nvars:]
    s12 = sigma[0:nvars, nvars:]
    s21 = sigma[nvars:, 0:nvars]
    
    # Solution of generalized eigenproblems.
    aux_1 = linalg.solve(s22, s21)
    lama, a = linalg.eig(np.dot(s12, aux_1), s11)
    aux_2 = linalg.solve(s11, s12)
    lamb, b = linalg.eig(np.dot(s21, aux_2), s22)
    
    
    # Sort a.  
    idx = np.argsort(lama)
    a = a[:, idx]
    
    # Sort b.        
    idx = np.argsort(lamb)
    b = b[:, idx]
    
    # Canonical correlations.        
    rho = np.sqrt(np.real(lamb[idx]))
    
    
    # Normalize dispersions.  
    tmp1 = np.dot(np.dot(a.T,s11), a)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    a = np.multiply(a, tmp3)
    b = np.mat(b)
    tmp1 = np.dot(np.dot(b.T,s22), b)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    b = np.multiply(b, tmp3)
        
    # Assure positive correlation
    tmp = np.diag(np.dot(np.dot(a.T,s12), b))
    b = np.dot(b,np.diag(tmp / np.abs(tmp)))

    # Canonical and MAD variates
    U = np.dot(a.T , (dm[0:nvars, :] - means[0:nvars, np.newaxis]))    
    V = np.dot(b.T , (dm[nvars:, :] - means[nvars:, np.newaxis]))          
    MAD = U - V  
    
            
    # New weights.        
    var_mad = np.tile(np.mat(2 * (1 - rho)).T, (1, ngood))    
    chisqr = np.sum(np.multiply(MAD, MAD) / var_mad, 0)
    wt = np.squeeze(1 - np.array(stats.chi2._cdf(chisqr, nvars)))
    
        
    # Continue iteration.        
    delta = np.sum(np.abs(rho - oldrho))
    oldrho = rho
    iter += 1
    
# reshape to original image size, by including nodata pixels   
# actually reshaping to image size plus one to add chi-squared values
MADout = np.zeros((int(nvars+1), infys_df_c1_c.shape[0]))
MADout[0:nvars, gooddataidx] = MAD
MADout[nvars, gooddataidx] = chisqr

0


ZeroDivisionError: Weights sum to zero, can't be normalized

In [31]:
# check whether adding chi-squared worked -> should be 9 rows (or in pyhton count: 0-8)
MADout_df = pd.DataFrame(MADout)
MADout_df.head(10)

,0,1,2,3,4,5,6,7,8
0,2.229130,-3.643466,-0.303445,-0.598453,-3.332359,-0.639299,0.328940,0.054010,2.773893e+07
1,0.496179,-0.066652,-0.197869,-0.151194,-0.028284,-0.046595,-0.012453,0.012687,1.485607e+06
2,-0.552385,1.990839,0.042888,0.687904,-0.192895,0.038691,0.221659,0.412802,1.570246e+09
3,2.745066,-1.183161,-0.733123,-0.181764,-0.748339,-0.188165,-0.128263,0.142595,1.874429e+08
4,-1.415857,2.824227,-0.710167,0.361194,0.297893,-0.178653,-0.309809,-0.384957,1.365783e+09
5,0.459397,-1.329931,0.191279,-0.199364,0.200009,0.221091,0.124848,-0.256200,6.048693e+08
6,0.523804,-2.872222,2.851455,1.778197,-0.450757,-1.587541,1.911392,-1.793369,2.964885e+10
7,1.556171,-2.330659,-0.583967,-0.298606,-0.551366,-0.504102,0.168449,0.160286,2.370398e+08
8,1.399868,0.661610,0.158005,-0.256975,0.014488,0.159734,0.042753,0.183625,3.106937e+08
9,2.320247,-0.419657,-0.301652,0.275931,-0.643236,0.001470,0.078643,0.519953,2.490940e+09


In [10]:
# increase dimension of transposed dataframe by adding on column called chi_squared
vars_with_chi_squared = vars + ["chi_squared"]

# transpose matrix 
MADout = np.transpose(MADout).copy()
MADout_df = pd.DataFrame(MADout, columns=vars_with_chi_squared)
# should now have 9 columns and the last one should be chi_squared
MADout_df.head(10)

,SpeciesCount,TreeCount,J,AvgTreeHeight,AvgDbh,AvgCrownDiameter,AvgCrownHeight,AvgCrownArea,chi_squared
0,2.229130,-3.643466,-0.303445,-0.598453,-3.332359,-0.639299,0.328940,0.054010,2.773893e+07
1,0.496179,-0.066652,-0.197869,-0.151194,-0.028284,-0.046595,-0.012453,0.012687,1.485607e+06
2,-0.552385,1.990839,0.042888,0.687904,-0.192895,0.038691,0.221659,0.412802,1.570246e+09
3,2.745066,-1.183161,-0.733123,-0.181764,-0.748339,-0.188165,-0.128263,0.142595,1.874429e+08
4,-1.415857,2.824227,-0.710167,0.361194,0.297893,-0.178653,-0.309809,-0.384957,1.365783e+09
5,0.459397,-1.329931,0.191279,-0.199364,0.200009,0.221091,0.124848,-0.256200,6.048693e+08
6,0.523804,-2.872222,2.851455,1.778197,-0.450757,-1.587541,1.911392,-1.793369,2.964885e+10
7,1.556171,-2.330659,-0.583967,-0.298606,-0.551366,-0.504102,0.168449,0.160286,2.370398e+08
8,1.399868,0.661610,0.158005,-0.256975,0.014488,0.159734,0.042753,0.183625,3.106937e+08
9,2.320247,-0.419657,-0.301652,0.275931,-0.643236,0.001470,0.078643,0.519953,2.490940e+09


In [11]:
# add coordinates
MADout_df["X"] = infys_df_c1["X"]
MADout_df["Y"] = infys_df_c1["Y"]
MADout_df.head(10)

,SpeciesCount,TreeCount,J,AvgTreeHeight,AvgDbh,AvgCrownDiameter,AvgCrownHeight,AvgCrownArea,chi_squared,X,Y
0,2.229130,-3.643466,-0.303445,-0.598453,-3.332359,-0.639299,0.328940,0.054010,2.773893e+07,-92.433389,15.588935
1,0.496179,-0.066652,-0.197869,-0.151194,-0.028284,-0.046595,-0.012453,0.012687,1.485607e+06,-99.546778,26.072916
2,-0.552385,1.990839,0.042888,0.687904,-0.192895,0.038691,0.221659,0.412802,1.570246e+09,-88.688611,20.654722
3,2.745066,-1.183161,-0.733123,-0.181764,-0.748339,-0.188165,-0.128263,0.142595,1.874429e+08,-102.826388,20.825555
4,-1.415857,2.824227,-0.710167,0.361194,0.297893,-0.178653,-0.309809,-0.384957,1.365783e+09,-104.218713,20.403546
5,0.459397,-1.329931,0.191279,-0.199364,0.200009,0.221091,0.124848,-0.256200,6.048693e+08,-108.957083,27.390907
6,0.523804,-2.872222,2.851455,1.778197,-0.450757,-1.587541,1.911392,-1.793369,2.964885e+10,-88.903333,19.444444
7,1.556171,-2.330659,-0.583967,-0.298606,-0.551366,-0.504102,0.168449,0.160286,2.370398e+08,-90.773055,18.278055
8,1.399868,0.661610,0.158005,-0.256975,0.014488,0.159734,0.042753,0.183625,3.106937e+08,-105.534166,24.736666
9,2.320247,-0.419657,-0.301652,0.275931,-0.643236,0.001470,0.078643,0.519953,2.490940e+09,-108.387500,30.217499


In [12]:
# create .csv
MADout_df.to_csv('iMAD_results_12_SEF.csv', index=False)

In [ ]:
#### ENDE ####

In [54]:
## Beta

MADout_df["Y"] = infys_df_c1["std_chg"]
np.std(madout(lstone))
madout(lastone <2*np.std)=2


# Add chi squared variable to table

CHIsqr_df = pd.DataFrame(chisqr)


chisqr = np.transpose(chisqr).copy()
CHI_df = pd.DataFrame(chisqr)

MADout_df['CHI'] = CHI_df



# Assuming chisqr is a 1D numpy array with length equal to the number of rows in MADout
# We ensure chisqr is reshaped as a 2D column vector if it isn't already
chisqr_column = chisqr.reshape(-1, 1)

# Append chisqr as a new column to MADout
MADout = np.concatenate((MADout, chisqr_column), axis=1)




MADout_df.head()



SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (54363561.py, line 5)